In [1]:
import sys
from pathlib import Path
import json
import numpy as np  
import copy

parent_dir = Path("/home/ujx4ab/ondemand/CBM_Final_Project/src/LLM")
sys.path.append(str(parent_dir))
parent_dir = Path("/home/ujx4ab/ondemand/CBM_Final_Project/src/Util")
sys.path.append(str(parent_dir))

from data_utils import load_reddit_posts, get_comments_for_post, get_user_comment_history, get_comments_in_post_thread, upload_json_to_file
from Mistral.minstral_utils import generate_mistral_prompt_zero_shot
from posttree import PostTree, UserCommentHistories


/home/ujx4ab/ondemand/CBM_Final_Project/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from minstral_utils import load_quantized_model, initialize_tokenizer
model_name = 'mistralai/Mistral-7B-Instruct-v0.3'
model = load_quantized_model(model_name)
tokenizer = initialize_tokenizer(model_name)

def run_prompt_on_mistral(prompt):
    stop_token_ids = [0]
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded[0].split("[/INST]", 1)[1].strip()
        

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [00:17<00:00,  5.72s/it]


In [3]:
post_tree = PostTree(
    '/home/ujx4ab/ondemand/CBM_Final_Project/Data/arcticshift/processed/r_books_posts.csv',
    '18vuw2v'
    )
post_tree.create_comment_tree('/home/ujx4ab/ondemand/CBM_Final_Project/Data/arcticshift/cleaned/r_books_comments_cleaned_and_pruned.csv')

# tree to be updated with generated comments
new_tree = copy.deepcopy(post_tree)

user_histories = UserCommentHistories(
    '/home/ujx4ab/ondemand/CBM_Final_Project/Data/arcticshift/cleaned/r_books_comments_cleaned.csv',
    '18vuw2v'
    )

list_lengths = [len(lst) for lst in user_histories.user_histories.values()]
mean_length = np.mean(list_lengths)
std_length = np.std(list_lengths)
print(f"Mean length: {mean_length}")
print(f"Standard deviation: {std_length}")


Mean length: 10.172043010752688
Standard deviation: 25.24444947229867


In [5]:
for node in post_tree.bfs_generator():
    # get the current comment and 10 of their history comments selected at random (if they have 10)
    cur_user_comment_history = user_histories.get_random_user_history(f"{node.user}")

    # get  previous responses on thread
    previous_comments = node.get_previous_responses()

    # get comment to be responded to and delete from response history
    try: 
        if len(previous_comments) > 0:
            comment_of_interest = previous_comments[-1]
        else:
            comment_of_interest = ""
        previous_comments = "\n".join(previous_comments)
    except TypeError: 
        comment_of_interest = ""
        previous_comments = ""

    prompt = generate_mistral_prompt_zero_shot(
        post_title=post_tree.title, 
        post_content=post_tree.content, 
        previous_comments_on_thread=previous_comments, 
        comment_of_interest=comment_of_interest,
        user_comment_history=cur_user_comment_history, 
    )

    response = run_prompt_on_mistral(prompt)
    node.comment_text = response
    
    print(response)    

post_tree.save_as_json(f"/home/ujx4ab/ondemand/CBM_Final_Project/Data/posttrees/{post_tree.post_id}.json")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/ujx4ab/ondemand/CBM_Final_Project/.venv/lib64/python3.11/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The user seems to appreciate thoughtful, well-researched works, appreciating the details and nuances in the writing. In response to the original post, the user might enjoy expanding on the comparison between Barbara Kingsolver's work and another author:

     "While I do genuinely enjoy Barbara Kingsolver's writing, I must admit that I've found Kim Stanley Robinson's work particularly captivating due to its meticulous attention to detail, especially the environmental and societal aspects. In comparison, I find that Kingsolver's work, while engaging and transportive, often lacks the same level of intricate detail in world-building. I'd be curious to know your thoughts on Robinson's work in this regard."

     This response brings a critical perspective, while still acknowledging the user's love for Barbara Kingsolver and inviting discussion.</s>


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"It's great to see your enthusiasm for Barbara Kingsolver's work! Her ability to transport readers is truly a gift. I haven't had the chance to read 'Lacuna' yet, but it's definitely up next on my list after returning from my holiday. I've heard it's equally as captivating as her other novels.

I can understand why the book 'Demon Copperhead' resonated with your mother so strongly, given Kingsolver's unique storytelling style that allows us to empathize with a wide range of characters. The recommendation of 'Poisonwood Bible' seems spot on too.

For those looking for additional recommendations, I'd suggest 'The Poisonwood Bible' by Barbara Kingsolver, as well as 'Love's Executioner and Other Tales of Psychotherapy' by Irving Yalom. I personally found 'Love's Executioner' to be a


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

"Have you also read the novel 'Prodigal Summer'? I haven't read it yet, but the premise intrigues me. It seems like a departure from Barbara Kingsolver's other works, as it focuses on the lives and ecological challenges faced by three separate communities in rural Tennessee. I'm curious to know if you found this book as absorbing as her other novels and how it compares to her other works."</s>


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"I completely agree with your insights on Barbara Kingsolver's genius. Her ability to transport readers is truly remarkable. I am eager to read more of her work as well, and I would highly recommend 'Lacuna' as your next read. It's an intriguing title, and I'm sure it will live up to the caliber of her other works. Keep sharing your love for Kingsolver; every voice that contributes to her continued acclaim is a welcome one!"</s>


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[Assistant's response]: "I wholeheartedly agree with your sentiments about Barbara Kingsolver's genius! Demon Copperhead was an incredible read, and I must say, so was Poisonwood Bible. If you haven't already, I highly recommend you read Lacuna as well. It's been a delight diving deeper into her work, and I can't wait to see what else she creates!"

    [Possible user response]: "Oh no, I haven't read Lacuna yet! Maybe after my holiday, I'll finally get to it. I'm so glad I found your post, as it's reinforced my love for Barbara Kingsolver's writing. I'm excited to see what else she comes up with!"</s>
